In [2]:
import os
os.makedirs("processed", exist_ok=True)


In [4]:
print(df.dtypes)

time        object
SensorA    float64
SensorB    float64
SensorC    float64
dtype: object


In [5]:
df = df.select_dtypes(include=[np.number])

In [6]:
print(df.isnull().sum())
df.fillna(method='ffill', inplace=True)  # forward fill NaN


SensorA    137
SensorB    235
SensorC     48
dtype: int64


In [7]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler()
X = df.iloc[:, :-1].values  # all features except target
y = df.iloc[:, -1].values   # last column = RUL

# Ensure numeric and no NaN
X = X.astype(float)

X_scaled = scaler.fit_transform(X)
print("✅ Scaling done, shape:", X_scaled.shape)

✅ Scaling done, shape: (1440, 2)


In [9]:
# milestone4_full_ready.py
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import joblib

# ------------------ CONFIG ------------------
fd_number = 1
window_size = 30
data_file = "sensor_data.csv"

processed_dir = "processed"
model_dir = "models_m2"
results_dir = "results"
os.makedirs(processed_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

model_path = os.path.join(model_dir, f"optimized_fd{fd_number}_milestone4.h5")
scaler_path = os.path.join(model_dir, f"scaler_fd{fd_number}_milestone4.save")

# ------------------ LOAD CSV ------------------
df = pd.read_csv(data_file)

# Keep only numeric columns
df = df.select_dtypes(include=[np.number])
df.fillna(method='ffill', inplace=True)  # forward fill NaN

# Features and target
X = df.iloc[:, :-1].values.astype(float)
y = df.iloc[:, -1].values.astype(float)

# ------------------ SCALE ------------------
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Save scaler
joblib.dump(scaler, scaler_path)
print(f"✅ Scaler saved at {scaler_path}")

# ------------------ CREATE SLIDING WINDOWS ------------------
X_lstm = []
y_lstm = []

for i in range(len(X_scaled) - window_size):
    X_lstm.append(X_scaled[i:i+window_size, :])
    y_lstm.append(y[i+window_size-1])

X_lstm = np.array(X_lstm)
y_lstm = np.array(y_lstm)

# Split into train/validation (80%-20%)
split_idx = int(0.8 * len(X_lstm))
X_train, y_train = X_lstm[:split_idx], y_lstm[:split_idx]
X_val, y_val = X_lstm[split_idx:], y_lstm[split_idx:]

print(f"✅ Data ready: X_train={X_train.shape}, X_val={X_val.shape}")

# ------------------ BUILD LSTM MODEL ------------------
model = Sequential([
    LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    BatchNormalization(),
    Dropout(0.2),
    
    LSTM(64, return_sequences=False),
    BatchNormalization(),
    Dropout(0.2),
    
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
model.summary()

# ------------------ CALLBACKS ------------------
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True)
]

# ------------------ TRAIN MODEL ------------------
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

print(f"✅ Model saved at {model_path}")

# ------------------ EVALUATE ------------------
y_val_pred = model.predict(X_val).flatten()
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
mae = mean_absolute_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)
accuracy = 100 - (rmse / np.max(y_val) * 100)

print(f"\n🔹 Model Evaluation — FD00{fd_number}")
print(f"RMSE : {rmse:.4f}")
print(f"MAE  : {mae:.4f}")
print(f"R² Score : {r2:.4f}")
print(f"Approx. Accuracy : {accuracy:.2f}%")

# ------------------ SAVE METRICS ------------------
results_df = pd.DataFrame({
    "FD_Set": [f"FD00{fd_number}"],
    "RMSE": [rmse],
    "MAE": [mae],
    "R2_Score": [r2],
    "Accuracy(%)": [accuracy]
})
results_df.to_csv(os.path.join(results_dir, f"model_performance_fd{fd_number}_milestone4.csv"), index=False)
print(f"📊 Metrics saved to {results_dir}")

# ------------------ ALERTS ------------------
warning_threshold = 50
critical_threshold = 20

alerts = []
for i, rul in enumerate(y_val_pred):
    if rul <= critical_threshold:
        alerts.append((i, rul, "CRITICAL"))
    elif rul <= warning_threshold:
        alerts.append((i, rul, "WARNING"))

alert_df = pd.DataFrame(alerts, columns=["Sample_Index", "Predicted_RUL", "Alert_Level"])
alert_df.to_csv(os.path.join(results_dir, f"alerts_fd{fd_number}_milestone4.csv"), index=False)
print(f"📢 Alerts saved to {results_dir}")
print(alert_df.head(10))


✅ Scaler saved at models_m2\scaler_fd1_milestone4.save
✅ Data ready: X_train=(1128, 30, 2), X_val=(282, 30, 2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 128)           67072     
                                                                 
 batch_normalization (Batch  (None, 30, 128)           512       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 30, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 batch_normalization_1 (Bat  (None, 64)                256       
 chNormalization)                                                
          

C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


18/18 [==============================] - 2s 108ms/step - loss: 15.1355 - mae: 3.4924 - val_loss: 49.8833 - val_mae: 6.9091
Epoch 3/100
18/18 [==============================] - 2s 104ms/step - loss: 5.2877 - mae: 1.8517 - val_loss: 28.8081 - val_mae: 5.1756
Epoch 4/100
18/18 [==============================] - 2s 101ms/step - loss: 2.9987 - mae: 1.3678 - val_loss: 26.9735 - val_mae: 4.9939
Epoch 5/100
18/18 [==============================] - 2s 94ms/step - loss: 3.1708 - mae: 1.4160 - val_loss: 28.3345 - val_mae: 5.1214
Epoch 6/100
18/18 [==============================] - 2s 98ms/step - loss: 2.8034 - mae: 1.3533 - val_loss: 19.9932 - val_mae: 4.2245
Epoch 7/100
18/18 [==============================] - 2s 98ms/step - loss: 3.0127 - mae: 1.3970 - val_loss: 19.2527 - val_mae: 4.1369
Epoch 8/100
18/18 [==============================] - 2s 113ms/step - loss: 2.9118 - mae: 1.3740 - val_loss: 18.3804 - val_mae: 4.0129
Epoch 9/100
18/18 [==============================] - 2s 110ms/step - loss: 2